In [1]:
%run TriggerFunction.ipynb
%run TargetFunction.ipynb
%run EffectFunction.ipynb

In [16]:
def make_abl_list(owner, trigger, bywhom, trigger_cond, target, n_targets, target_pos, 
                 effect, effect_from, effect_info, effect_repeat, n_charges):
    # if multiple triggers, make multiple abilities
    if len(trigger) > 1:
        assert len(trigger) == len(bywhom) == len(trigger_cond), \
            'if abl has multiple triggers, it needs multiple bywhoms and trigger_conds as well'
        abl_list = [Ability(owner, trigger, bywhom, trigger_cond, target, n_targets, target_pos, 
                            effect, effect_from, effect_info, effect_repeat, n_charges) 
                    for trigger, bywhom, trigger_cond in zip(trigger, bywhom, trigger_cond)]
        return abl_list
        
    else:
        return [Ability(owner, trigger[0], bywhom[0], trigger_cond, target, n_targets, target_pos, 
                            effect, effect_from, effect_info, effect_repeat, n_charges)]
    
class Ability():
    def __init__(self, owner, trigger, bywhom, trigger_cond, 
                 target, n_targets, target_pos, 
                 effect, effect_from, effect_info, effect_repeat, 
                 n_charges = 999):
        self.owner = owner
        self.trigger = trigger
        self.trigger_bywhom = bywhom
        self.target = ''.join([t.__name__ for t in target])
        self.effect = ''.join([e.__name__ for e in effect])
        self.triggerers = []
        self.n_charges = n_charges
        self.n_charges_ref = n_charges
        
        # trigger
        self.trigger_func = TriggerFunction(self.owner, trigger, bywhom, trigger_cond, self.target == 'Triggerer')
        
        # target
        if len(target) > 1:
            assert isinstance(target_pos, list), 'if ability has multiple targets, target_pos must be list of tuples'
            self.target_funcs = [t(self.owner, n_targets, tp) for t, tp in zip(target, target_pos)]
        else:
            self.target_funcs = [t(self.owner, n_targets, target_pos) for t in target]
        
        # effect
        if len(effect) > 1:
            self.effect_funcs = [e(self.owner, ef, ei, effect_repeat) for e, ef, ei in zip(effect, effect_from, effect_info)]
        else:
            self.effect_funcs = [e(self.owner, ef, ei, effect_repeat) for e, ef, ei in zip(effect, effect_from, [effect_info])]
                
    def __repr__(self):
        return f'{self.trigger_func.name}: {self.owner.species} do ' + \
               f'{self.effect} on {self.target}'
    
    def use(self, gamestate, in_team1, verbose=False):
        targets_list = [target_func(gamestate, in_team1, self.triggerers) for target_func in self.target_funcs]
        for targets in targets_list:
            assert isinstance(targets, list), f'target_func must return list but instead got: {target}'
            for target in targets:
                assert isinstance(target, (Animal, tuple)) , f'targets must contain lists of animals but instead had: {target}'
                
        # if no charges left
        if self.n_charges <= 0:
            if verbose: 
                print(f'{self.owner} tried to {self.effect} but had no remaining charges.')
            return False
        self.n_charges -= 1
        
        # if no targets provided
        if all([len(targets) == 0 for targets in targets_list]) and 'NoTarget' not in self.target and self.effect != 'Summon':
            if verbose: 
                print(f'{self.owner} tried to {self.effect} but had no targets.')
            return False
        
        # if owner is dead and should not use ability
        # TODO: new update removes StartOfBattle clause
        elif self.owner.hp <= 0 and self.trigger != 'StartOfBattle' and self.trigger_bywhom + self.trigger != 'SelfFaint':
            if verbose: 
                print(f'{self.owner.species} tried to {self.effect} but was already dead.')
            return False
            
        # else, use ability
        for targets, effect_func in zip(targets_list, self.effect_funcs):
            # custom triggerer
            if self.target == 'Triggerer':
                for pet, n_triggers in self.triggerers:
                    for _ in range(n_triggers):
                        effect_func([pet], gamestate, in_team1)
                    if verbose: 
                        print(f'{self.owner.species} used {effect_func.__class__.__name__} on {pet.species}, ' + \
                              f'{n_triggers} time(s).')
            else:
                effect_func(targets, gamestate, in_team1)
                if verbose: 
                    print(f'{self.owner.species} used {effect_func.__class__.__name__} on ' + \
                          f'{[pet.species for pet in targets]}.')
                    
        
        

        